In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))

In [18]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from utilities.training import train, confusion
from elements.abstracts import AbstractModulator
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
from pickle import dump, load
import torch
import numpy
import timm
import pandas
from copy import deepcopy

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA A100-SXM4-80GB


In [19]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=False,  in_chans=1, num_classes=classes).to(device)

In [20]:
def combined_loss(cross_entropy_to_mse_proportion:float=1.0):
    def loss_function(outputs, targets):
        CELoss = torch.nn.functional.cross_entropy(outputs, targets)
        MSELoss = torch.nn.functional.mse_loss(torch.nn.functional.softmax(outputs, dim=1), torch.nn.functional.one_hot(targets, num_classes=10).float())
        loss = cross_entropy_to_mse_proportion*CELoss + (1.0-cross_entropy_to_mse_proportion)*MSELoss
        return loss
    return loss_function

In [21]:
# Предпочтительные параметры
size = 30.0E-6
near_N = 1336 #1336
near_length = 10.0E-3
wavelength = 500.0E-9
detectors_amount = 24
masks_amount = 3
distance = 0.09052

# Параметры обучения
batch_size = 50
learning_rate = 0.009854
loss_function_proportion = 0.609798
optimizer_type_name = 'RMSprop'
optimizer_types_list = {'Adam':torch.optim.Adam, 'SGD':torch.optim.SGD, 'RMSprop':torch.optim.RMSprop, 'Adagrad':torch.optim.Adagrad}

# Параметры когерентности
spatial_coherence = 100.0E-6
time_coherence = 10.0E-9
time_relaxation = 1.0E-6
mean_samples = 100

# Вычисляемые параметры
pixels = upper_integer(near_length/size)
length = pixels * size
cppp = upper_integer(near_N * size / length)
N = upper_integer(length*cppp / size) 
detector_size = length / 60

print(f"Длинна волны:                             {engineering(wavelength, 'м')}")
print(f"Количество вычислительных пикселей:       {N}")
print(f"Количество пикселей маски:                {pixels}")
print(f"Размер оптических элементов:              {engineering(length, 'м')}")
print(f"Размер пикселя маски:                     {engineering(length/pixels, 'м')}")
print(f"Расстояние между слоями:                  {engineering(distance, 'м')}")

print(f"Временная когерентность:                  {engineering(time_coherence, 'с')}")
print(f"Время релаксации:                         {engineering(time_relaxation, 'c')}")
print(f"Пространственная когерентность:           {engineering(spatial_coherence, 'м')}")
print(f"Количество усреднений:                    {mean_samples}")

print(f"Размер детекторов:                        {engineering(detector_size, 'м')}")
print(f"Количество детекторов:                    {detectors_amount} на {detectors_amount}")

print(f"Пропорция CE к MSE лосс:                  {loss_function_proportion}")
print(f"Размер батча:                             {batch_size}")
print(f"Тип оптимизатора:                         {optimizer_type_name}")

Длинна волны:                             500.0 нм
Количество вычислительных пикселей:       1336
Количество пикселей маски:                334
Размер оптических элементов:              10.02 мм
Размер пикселя маски:                     30.0 мкм
Расстояние между слоями:                  90.52 мм
Временная когерентность:                  10.0 нс
Время релаксации:                         1.0 мкc
Пространственная когерентность:           100.0 мкм
Количество усреднений:                    100
Размер детекторов:                        167.0 мкм
Количество детекторов:                    24 на 24
Пропорция CE к MSE лосс:                  0.609798
Размер батча:                             50
Тип оптимизатора:                         RMSprop


In [34]:
spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
detectors_filter = Gaussian((detector_size, detector_size), (0,0))

dataset = Dataset('CIFAR10', batch_size, N, N, torch.complex64)
dataset.train
dataset.test

Files already downloaded and verified
Files already downloaded and verified


In [47]:
propagation         = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
phase_modulator     = PhaseModulator(N, length, pixels)
amplitude_modulator = AmplitudeModulator(N, length, pixels)
detectors = MatrixDetectors(N, length, wavelength, detectors_amount, detectors_filter, spectral_filter).to(device)
electronic = resnet18()

propagation = propagation.to(device)
phase_modulator = phase_modulator.to(device)
amplitude_modulator = amplitude_modulator.to(device)
detectors = detectors.to(device)
electronic = electronic.to(device)

propagation = torch.nn.DataParallel(propagation)
phase_modulator = torch.nn.DataParallel(phase_modulator)
amplitude_modulator = torch.nn.DataParallel(amplitude_modulator)
detectors = torch.nn.DataParallel(detectors)
electronic = torch.nn.DataParallel(electronic)

In [50]:
field, _ = next(iter(dataset.train))
field = field.to(device)
field = phase_modulator(field)

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/maindev/.local/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/home/maindev/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/maindev/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/maindev/OpticalEncoder/elements/abstracts.py", line 251, in forward
    field = field * self._multiplier()
RuntimeError: The size of tensor a (2) must match the size of tensor b (1336) at non-singleton dimension 4


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

# Определение моделей
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.layer = nn.Linear(10000, 20000)

    def forward(self, x):
        return self.layer(x)

class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.layer = nn.Linear(20000, 30000)

    def forward(self, x):
        return self.layer(x)

# Инициализация моделей
model1 = Model1()
model2 = Model2()

# Перемещение моделей на доступные GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = model1.to(device)
model2 = model2.to(device)

# Использование DataParallel для автоматического распределения по GPU
if torch.cuda.device_count() > 1:
    model1 = nn.DataParallel(model1)
    model2 = nn.DataParallel(model2)

for i in range(100):
    # Пример данных
    data = torch.randn(64, 10000).to(device)  # Батч размером 64, каждый с вектором размером 10
    
    # Оптимизация и критерий потерь
    criterion = nn.MSELoss()
    optimizer = optim.SGD(list(model1.parameters()) + list(model2.parameters()), lr=0.01)
    
    # Обработка данных первой моделью
    output1 = model1(data)
    
    # Обработка данных второй моделью
    output2 = model2(output1)
    
    # Пример цели
    target = torch.randn(64, 30000).to(device)
    loss = criterion(output2, target)
    loss.backward()
    optimizer.step()
    
    print(f"Loss: {loss.item()}")

Loss: 1.1088625192642212
Loss: 1.1102659702301025
Loss: 1.1119208335876465
Loss: 1.110910177230835
Loss: 1.1119478940963745
Loss: 1.1111277341842651
Loss: 1.1117554903030396
Loss: 1.1104594469070435
Loss: 1.110807180404663
Loss: 1.1118454933166504
Loss: 1.110033631324768
Loss: 1.111470103263855
Loss: 1.111626386642456
Loss: 1.1086947917938232
Loss: 1.1129156351089478
Loss: 1.1087734699249268
Loss: 1.1122708320617676
Loss: 1.1119248867034912
Loss: 1.1099510192871094
Loss: 1.1092486381530762
Loss: 1.1098417043685913
Loss: 1.1106255054473877
Loss: 1.111372947692871
Loss: 1.1087270975112915
Loss: 1.113295316696167
Loss: 1.113590121269226
Loss: 1.1090023517608643
Loss: 1.1110398769378662
Loss: 1.1111435890197754
Loss: 1.1094117164611816
Loss: 1.1100033521652222
Loss: 1.1102880239486694
Loss: 1.1113914251327515
Loss: 1.109862208366394
Loss: 1.1108981370925903
Loss: 1.11057710647583
Loss: 1.1093593835830688
Loss: 1.1105353832244873
Loss: 1.1086257696151733
Loss: 1.1099516153335571
Loss: 1.112